In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
import pandas as pd
import numpy as np

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0'
}
response = requests.get('https://uk.trustpilot.com/review/toogoodtogo.com?page=2', headers=headers)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content)
data = json.loads(soup.find('script', type='application/json').string)
reviews = data["props"]["pageProps"]["reviews"]

sample_reply = reviews[0]
sample_reply

{'id': '61914c4101034f34babf3c56',
 'filtered': False,
 'text': 'Dissapointed with my first box. It was from Morrisons It contained 3 strawberries,  2 apples, 2 bags of carrots .4 bags of wonky  mixed peppers and 4 bags of runner beans which were not fit for consumption. and 6 oranges. I doubt it would have cost £10.',
 'rating': 1,
 'labels': {'merged': None,
  'verification': {'isVerified': False,
   'createdDateTime': '2021-11-14T17:49:53.000Z',
   'reviewSourceName': 'Organic',
   'verificationSource': 'invitation',
   'verificationLevel': 'not-verified'}},
 'title': 'Dissapointed with my first box',
 'likes': 0,
 'dates': {'experiencedDate': None,
  'publishedDate': '2021-11-14T17:49:53.000Z',
  'updatedDate': '2021-11-14T17:50:30.000Z'},
 'report': None,
 'hasUnhandledReports': False,
 'consumer': {'id': '61914c343f59d00012152fb3',
  'displayName': 'Yvonne Simpson',
  'imageUrl': '',
  'numberOfReviews': 1,
  'countryCode': 'GB',
  'hasReviewFacebookFriend': False,
  'isDirectFac

In [5]:
reviews[0]['dates']['publishedDate']

'2021-11-14T17:49:53.000Z'

In [6]:
#headers = {
   # 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0'
#}

#collecting all the info of the review we might need to process after checking it
usersname = []
reviewsText = []
reviewsTitle = []
ratings = []
country = []
language = []
publishedDate = []


for i in range(1,74):
    url = "https://uk.trustpilot.com/review/toogoodtogo.com?page={}".format(i)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    data = json.loads(soup.find('script', type='application/json').string)
    reviews = data["props"]["pageProps"]["reviews"]
    for review in reviews:
        #usersname.append(review['consumer']['displayName'])
        reviewsTitle.append(review['title'])
        reviewsText.append(review['text'])
        ratings.append(review['rating'])
        country.append(review['consumer']['countryCode'])
        language.append(review['language'])
        publishedDate.append(review['dates']['publishedDate'])




#response.status_code

In [7]:
#more readable in a dataframe
review_data = pd.DataFrame(np.column_stack([ratings, country, publishedDate, reviewsTitle, reviewsText, language]), 
                               columns=['rating', 'location', 'date', 'title', 'content', 'language'])

In [8]:
review_data

,name,rating,location,date,title,content,language
0,Elle S,5,GB,2021-12-01T21:47:43.000Z,What an amazing and innovative idea,What an amazing and innovative idea. I have or...,en
1,A K,5,GB,2021-12-01T13:18:28.000Z,So nice bakery lovely food thanks…,So nice bakery lovely food thanks bristanbul,en
2,Mark Arnold,1,GB,2021-11-28T14:47:29.000Z,No confidence in app,Reserved food on two occasions at Toby Inns at...,en
3,Claire,5,GB,2021-11-28T10:15:04.000Z,Great app and such a good idea!,Updated -\n\nOne of the best app ideas around....,en
4,Lewi Lewis,2,GB,2021-11-26T23:28:59.000Z,An Excellent Idea Often Poorly Executed By Par...,I've had good and bad experiences after using ...,en
...,...,...,...,...,...,...,...
1455,Caroline Johnson,5,GB,2016-08-23T20:49:42.000Z,Great food,I really enjoyed the food I got at the fish an...,en
1456,Wheelys Cph,5,DK,2016-08-17T19:21:12.739Z,Good concpt,Nice food and very nice people,en
1457,Faith Tidy,5,GB,2016-08-11T14:15:44.654Z,Too Good To Go,"Really great idea, and company are very respon...",en
1458,Afif Hossain Emon,5,DK,2016-08-02T15:23:00.000Z,Supporting a cause,Good job. So far work perfect :),en


In [9]:
#to check if all the reviews are in english
review_data['language'].value_counts()

en    1460
Name: language, dtype: int64

In [10]:
review_data['location'].value_counts()

GB    1110
DK     117
NO      44
US      34
DE      31
NL      31
CH      19
BE      19
FR      17
ES       5
SE       5
AU       4
IT       3
CA       3
GR       3
PT       2
PL       2
MY       2
IE       2
IN       1
AT       1
BN       1
UA       1
RO       1
AW       1
LU       1
Name: location, dtype: int64

In [12]:
df = review_data[review_data['location']=='GB']

In [13]:
df['review_content'] = df['title'] + ' ' + df['content']
df

/tmp/ipykernel_4335/2801066844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_content'] = df['title'] + ' ' + df['content']


,name,rating,location,date,title,content,language,review_content
0,Elle S,5,GB,2021-12-01T21:47:43.000Z,What an amazing and innovative idea,What an amazing and innovative idea. I have or...,en,What an amazing and innovative idea What an am...
1,A K,5,GB,2021-12-01T13:18:28.000Z,So nice bakery lovely food thanks…,So nice bakery lovely food thanks bristanbul,en,So nice bakery lovely food thanks… So nice bak...
2,Mark Arnold,1,GB,2021-11-28T14:47:29.000Z,No confidence in app,Reserved food on two occasions at Toby Inns at...,en,No confidence in app Reserved food on two occa...
3,Claire,5,GB,2021-11-28T10:15:04.000Z,Great app and such a good idea!,Updated -\n\nOne of the best app ideas around....,en,Great app and such a good idea! Updated -\n\nO...
4,Lewi Lewis,2,GB,2021-11-26T23:28:59.000Z,An Excellent Idea Often Poorly Executed By Par...,I've had good and bad experiences after using ...,en,An Excellent Idea Often Poorly Executed By Par...
...,...,...,...,...,...,...,...,...
1444,Cecile Perol,4,GB,2016-10-31T23:06:02.000Z,Good bag of selection of breads,Would have prefer more pastries (only 2 in bag...,en,Good bag of selection of breads Would have pre...
1451,Simon Champion,5,GB,2016-09-05T21:54:10.000Z,First try of TGTG at Moshimo Brighton,"I'd been meaning to give this app a try out, a...",en,First try of TGTG at Moshimo Brighton I'd been...
1454,David Choy,5,GB,2016-08-24T17:58:28.000Z,exactly as stated,"ordered food with app, picked it up, no proble...",en,"exactly as stated ordered food with app, picke..."
1455,Caroline Johnson,5,GB,2016-08-23T20:49:42.000Z,Great food,I really enjoyed the food I got at the fish an...,en,Great food I really enjoyed the food I got at ...


In [16]:
df = df.drop(columns=['location','title','content','language'])

In [17]:
df['source'] = 'Trustpilot'
df

,rating,date,review_content,source
0,5,2021-12-01T21:47:43.000Z,What an amazing and innovative idea What an am...,Trustpilot
1,5,2021-12-01T13:18:28.000Z,So nice bakery lovely food thanks… So nice bak...,Trustpilot
2,1,2021-11-28T14:47:29.000Z,No confidence in app Reserved food on two occa...,Trustpilot
3,5,2021-11-28T10:15:04.000Z,Great app and such a good idea! Updated -\n\nO...,Trustpilot
4,2,2021-11-26T23:28:59.000Z,An Excellent Idea Often Poorly Executed By Par...,Trustpilot
...,...,...,...,...
1444,4,2016-10-31T23:06:02.000Z,Good bag of selection of breads Would have pre...,Trustpilot
1451,5,2016-09-05T21:54:10.000Z,First try of TGTG at Moshimo Brighton I'd been...,Trustpilot
1454,5,2016-08-24T17:58:28.000Z,"exactly as stated ordered food with app, picke...",Trustpilot
1455,5,2016-08-23T20:49:42.000Z,Great food I really enjoyed the food I got at ...,Trustpilot


In [19]:
df = df[['review_content','rating','date','source']]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review_content  1110 non-null   object
 1   rating          1110 non-null   object
 2   date            1110 non-null   object
 3   source          1110 non-null   object
dtypes: object(4)
memory usage: 43.4+ KB
